<a href="https://colab.research.google.com/github/kelifi/-Mask-Detection-and-Recognition/blob/main/Digit_recognation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Libraries**

In [1]:
import warnings
warnings.filterwarnings("ignore")

# Import the necessary libs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.datasets import mnist
import tensorflow as tf

sns.set(style='white', context='notebook', palette='deep')

# Load the data 

In [2]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/digit-recognizer/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/digit-recognizer/test.csv')
sub = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/digit-recognizer/sample_submission.csv')

# **Set data features and Target labels**

In [3]:
Y_train = train["label"]
X_train = train.drop(labels = ["label"], axis = 1) 

In [4]:
# Load more data sets, if there is no such data, validation accuracy = 0.9964
# With this batch of data, the validation accuracy can reach 0.9985
(x_train1, y_train1), (x_test1, y_test1) = mnist.load_data()

train1 = np.concatenate([x_train1, x_test1], axis=0)
y_train1 = np.concatenate([y_train1, y_test1], axis=0)

Y_train1 = y_train1
X_train1 = train1.reshape(-1, 28*28)

11501568/11490434 [==============================] - 0s 0us/step


## 2.2 Normalization
Moreover the CNN converg faster on [0..1] data than on [0..255].

In [5]:
# Normalize data to make CNN faster
X_train = X_train / 255.0
test = test / 255.0

X_train1 = X_train1 / 255.0

In [6]:
# Reshape Picture is 3D array (height = 28px, width = 28px , canal = 1)
X_train = np.concatenate((X_train.values, X_train1))
Y_train = np.concatenate((Y_train, Y_train1))

In [7]:
# Reshape image in 3 dimensions (height = 28px, width = 28px , canal = 1)
# canal = 1 => For gray scale
X_train = X_train.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

In [8]:
# Convert label to one hot vectors (ex : 2 -> [0,0,1,0,0,0,0,0,0,0])
Y_train = to_categorical(Y_train, num_classes = 10)

In [9]:
Y_train.shape

(112000, 10)

##  Split training and valdiation set

In [10]:
# Split the train and the validation set for the fitting
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=2)

# **Model Definition**

In [11]:
model = Sequential()

model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (28,28,1)))
model.add(BatchNormalization())

model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', activation ='relu'))
model.add(BatchNormalization())

model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(BatchNormalization())

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'Same',  activation ='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(10, activation = "softmax"))

In [12]:
# Compile model
import tensorflow as tf
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer = opt , loss = "categorical_crossentropy", metrics=["accuracy"])

In [13]:
epochs = 8
batch_size = 128

In [14]:
datagen = ImageDataGenerator(
        featurewise_center=False, # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

#datagen.fit(X_train)
train_gen = datagen.flow(X_train,Y_train, batch_size=batch_size)

## Model training

In [15]:
history = model.fit( X_train,Y_train,batch_size=128,epochs=10)

Epoch 1/10
788/788 [==============================] - 1607s 2s/step - loss: 0.1003 - accuracy: 0.9687
Epoch 2/10
788/788 [==============================] - 1607s 2s/step - loss: 0.0358 - accuracy: 0.9888
Epoch 3/10
788/788 [==============================] - 1609s 2s/step - loss: 0.0269 - accuracy: 0.9915
Epoch 4/10
788/788 [==============================] - 1594s 2s/step - loss: 0.0218 - accuracy: 0.9930
Epoch 5/10
788/788 [==============================] - 1599s 2s/step - loss: 0.0192 - accuracy: 0.9940
Epoch 6/10
788/788 [==============================] - 1595s 2s/step - loss: 0.0165 - accuracy: 0.9949
Epoch 7/10
788/788 [==============================] - 1602s 2s/step - loss: 0.0164 - accuracy: 0.9949
Epoch 8/10
788/788 [==============================] - 1612s 2s/step - loss: 0.0129 - accuracy: 0.9959
Epoch 9/10
788/788 [==============================] - 1610s 2s/step - loss: 0.0117 - accuracy: 0.9961
Epoch 10/10
788/788 [==============================] - 1634s 2s/step - loss: 0.011

In [16]:
# Make predictions about test sets
results = model.predict(test)

# Convert one-hot vector to number
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")

In [18]:
# Save the final result in cnn_mnist_submission.csv
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("cnn_mnist_submission2.csv",index=False)